In [1]:
import seedDataFetch

In [2]:
import numpy as numpy
import pandas
import scipy.stats as stats
data = seedDataFetch.GetRiotData('test.txt')

rawDataFrame = pandas.DataFrame(data)
outcomeVsFirstBlood = numpy.zeros([1,2])

wonGame = False
firstBlood = False
outcome = []
firstBloodList = []
totalFirstBlood = 0

totalGames = 0
firstObjectiveWins = {'Kill':0,'Tower':0,'Baron':0,'Dragon':0}
print(firstObjectiveWins.values())
firstObj=[0,0,0,0]
chi2Values=[]
firstObjective = 0
bins = numpy.array([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5,15.0])
wins = numpy.zeros([1,len(bins)])
totals = numpy.zeros([1,len(bins)])

print(numpy.digitize(.1,bins))
#inds = numpy.digitize(x, bins)
count = 0
notSupport = 0
for x in range(0,len(rawDataFrame.index)):
    for y in range(0,len(rawDataFrame.at[x,'matches'])):
        if rawDataFrame.at[x,'matches'][y]['gameMode'] == 'CLASSIC':
            if rawDataFrame.at[x,'matches'][y]['teams'][0]['win'] == 'Win':
                wonGame = True
            else:
                wonGame = False
            if wonGame == rawDataFrame.at[x,'matches'][y]['teams'][0]['firstBlood']:
                firstObjectiveWins['Kill'] += 1
            if wonGame == rawDataFrame.at[x,'matches'][y]['teams'][0]['firstTower']:
                firstObjectiveWins['Tower'] += 1
            if wonGame == rawDataFrame.at[x,'matches'][y]['teams'][0]['firstBaron']:
                firstObjectiveWins['Baron'] += 1
            if wonGame == rawDataFrame.at[x,'matches'][y]['teams'][0]['firstDragon']:
                firstObjectiveWins['Dragon'] += 1
        win = 'Win'
        csDeltaAt10 = 0
        for z in range(0,int(len(rawDataFrame.at[x,'matches'][y]['participants'])/2)):
            if rawDataFrame.at[x,'matches'][y]['participants'][z]['timeline']['role']!='DUO_SUPPORT':
                notSupport += 1
                if rawDataFrame.at[x,'matches'][y]['participants'][z]['teamId'] == rawDataFrame.at[x,'matches'][y]['teams'][0]['teamId']:
                    win = rawDataFrame.at[x,'matches'][y]['teams'][0]['win']
                else:
                    win = rawDataFrame.at[x,'matches'][y]['teams'][1]['win']
                if win == 'Win' and 'csDiffPerMinDeltas' in rawDataFrame.at[x,'matches'][y]['participants'][z]['timeline'].keys(): 
                    if rawDataFrame.at[x,'matches'][y]['participants'][z]['timeline']['csDiffPerMinDeltas']['0-10'] > 0:
                        csBin = numpy.digitize(rawDataFrame.at[x,'matches'][y]['participants'][z]['timeline']['csDiffPerMinDeltas']['0-10'], bins)
                        wins[0,csBin] += 1
                        totals[0,csBin] += 1
                    else:
                        csBin = numpy.digitize(abs(rawDataFrame.at[x,'matches'][y]['participants'][z]['timeline']['csDiffPerMinDeltas']['0-10']), bins)
                        totals[0,csBin] += 1        
                    count += 1
        totalGames += 1




chi2 = numpy.zeros([4,1])
p =numpy.zeros([4,1])
values = numpy.fromiter(firstObjectiveWins.values(), dtype = float)
for x in range(0,len(values)):
    chi2[x],p[x] = stats.chisquare([values[x]], [totalGames/2])


import matplotlib.pyplot as pl
# =============================================================================
# 
# Uncomment this code to make first plot, chi-squared values for first objective - win correlation
# 
# =============================================================================

xticks=['Kill', 'Tower', 'Baron', 'Dragon']
x=[1,2,3,4]
pl.xlim(0.5,4.5)
pl.ylim(10,85)
pl.plot(x,chi2, 'bo')
pl.xticks(x,firstObjectiveWins.keys())
pl.ylabel("Chi-squared", size=15)
pl.xlabel("First Objective Secured", size=15)
pl.title("Game First Objectives Relevance", size=20)

# =============================================================================
# 
# Uncomment this code to make second plot, win % against Cs/min deltas
# 
# =============================================================================

#winPercentage = numpy.divide(wins,totals)
#bins = numpy.reshape(bins, winPercentage.shape)
#bins = bins.reshape((10,))
#winPercentage = winPercentage.reshape((10,))
# 
#slope, intercept, r_value, p_value, std_err = stats.linregress(bins[:8],y=winPercentage[:8])
#pl.xlim(0,5)
#pl.ylim(0,1)
#pl.plot(bins,winPercentage, 'bo', label = 'Data')
#r2 = round((r_value*r_value),3)
#pl.ylabel("Win %", size=15)
#pl.xlabel("CreepScore/min $\Delta$, < 10 minutes", size=15)
#pl.plot(bins,bins*slope + intercept, 'r', label = 'Fit (r^2 = ' + str(r2) + ')')
#pl.legend()
#pl.show()


dict_values([0, 0, 0, 0])
0
